<a href="https://colab.research.google.com/github/rlagosb/taller_eiv/blob/main/0_Ejemplos_Anonimizaci%C3%B3n_Programacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Examinemos dos ejemplos de datos para la programación y apliquemos las orientaciones de la **[Norma técnica de anonimización para la publicación bases de datos como datos abiertos](https://repositoriodeis.minsal.cl/ContenidoSitioWeb2020/EstandaresNormativa/Norma%20t%C3%A9cnica%20241%20de%20anonimizaci%C3%B3n%20datos%20abiertos.pdf)**.  del Departamento de Estadísticas e Información de Salud (2025).


# Ejemplo 1: Fichas de Programación

En este script extraemos la programación de un conjunto de profesionales de un servicio clínico y guardamos los datos originales, seudoanonimizados y anonimizados



> ***Anonimización**: Procedimiento en virtud del cual un dato personal no puede
vincularse o asociarse a una persona determinada, ni permitir su  identificación, por haberse destruido o eliminado el nexo con la información que vincula, asocia o identifica a esa persona. Un dato anonimizado deja de ser un dato personal.*

> ***Seudoanonimización**: es el tratamiento de datos personales de manera tal que ya no puede atribuirse a un titular sin utilizar información adicional, siempre que dicha información adicional figure por separado y esté sujeta a medidas técnicas y organizativas destinadas a garantizar que los datos personales no se atribuyan a una persona física identificada o identificable. Este proceso es reversible, en cuanto que,al juntar información adicional con los datos personales seudoanonimizados, se podrá volver a atribuir ese dato a una persona identificada o identificable. Este proceso se denomina reidentificación.*

In [ ]:
#@title Descargar Planilla

import pandas as pd
import openpyxl as xl
import requests
import os

# Localización de los datos
path_fichas="https://raw.githubusercontent.com/rlagosb/taller_eiv/main/data/programacion/Fichas/"
planilla_fichas = 'Fichas Unidad Endoscopia.xlsx'

# Descargar el archivo
response = requests.get(path_fichas + planilla_fichas)
with open(planilla_fichas, 'wb') as f:
    f.write(response.content)

# Cargar planilla
wb = xl.load_workbook(filename = planilla_fichas, data_only=True)

print(f'Planilla {planilla_fichas} descargada ✅')

In [ ]:
#@title Obtener fichas

# Cargar datos de las fichas en las hojas de la planilla
fichas = {}
print('Cargando fichas:')

for ws_name in [name for name in wb.sheetnames if name not in ['Personal', 'Prestaciones', 'Ficha Programación']]:

  #Cargar hoja
  ws=wb[ws_name]

  #obtener datos profesional y eliminar cabecera
  profesional={'rut':ws['C4'].value,'nombre':ws['C3'].value,'profesion':ws['C5'].value,'obs_ficha':ws['F5'].value}
  ws.delete_rows(1,9)
  ws.delete_cols(1,1)

  #obtener actividades y horarios
  prog = (pd.DataFrame(ws.values).rename(columns={0:"ini",1:"fin"}).
          melt(id_vars=['ini','fin'], var_name='dia', value_name='act').
          # eliminar bloques sin actividades programadas
          dropna(subset=['act']))
  prog['dia']=prog['dia']-1

  #agregar datos profesional
  for col in ['rut','nombre','profesion','obs_ficha']:
    prog[col]=profesional[col]

  #agrupar bloques de actividades
  prog['id_prog']=((prog.rut!=prog.rut.shift()) |             # agrupar por cambio de rut
                    (prog.dia!=prog.dia.shift()) |            # por cambio de dia
                    (prog.act!=prog.act.shift())              # por cambio de actividad
                    ).cumsum()                                # asignar correlativo
  fichas[ws_name]=prog.groupby(['id_prog','rut','nombre','profesion','dia','act','obs_ficha'],as_index=False, dropna=False).aggregate({'ini':'min','fin':'max'})

  print(ws.title, '✔')



In [ ]:
#@title Tabular datos

#Consolidar  fichas en una tabla
programacion=pd.concat(fichas, ignore_index=True)

# Calcular duración actividades
programacion['hrs_prog']=(pd.to_datetime(programacion['fin'].astype(str), format='%H:%M:%S') -
                     pd.to_datetime(programacion['ini'].astype(str), format='%H:%M:%S')).dt.total_seconds() / 3600

programacion

🔴 ¿Cuáles campos son identificadores explícitos? ¿Cuasi-identificadores?

> • ***Identificadores explícitos**. Son datos que permiten identificar de forma inequívoca a una persona como el nombre, número de identificación nacional, [...] número de teléfono móvil.*

> • ***Cuasi-identificadores**. Son datos que no permiten una identificación directa del individuo, pero que en conjunto con otros datos pueden llegar a señalar a la persona como sexo, género, fecha de nacimiento, edad, ocupación, estado civil, nacionalidad, lugar de atención, comuna, región, previsión, fecha de egreso, entre otros.*

> • ***Atributos sensibles**: Son datos que revelan características físicas o morales y que pueden comprometer la privacidad de los individuos como los diagnósticos, procedimientos clínicos, estado de vacunación, entre otros.*

In [ ]:
#@title Des-identificación
# Cremos un identificador alternativo

profesionales = programacion[['rut']].drop_duplicates(ignore_index=True).reset_index().rename(columns={'index':'idProfesional'})
profesionales['idProfesional'] +=1
print('Identificadores de Profesionales\n', profesionales)

# Agregamos el pseudoidentificador y eliminamos los datos personales

programacion_desidentificada = programacion.merge(profesionales, on='rut', how='left').drop(columns=['rut','nombre','profesion'])
programacion_desidentificada

In [ ]:
# @title Anonimización
# Eliminamos los campos identificadores y pseudoidentificadores, dejando sólos los campos indispensables

programacion_anonimizada = programacion.groupby(['profesion','act'], dropna=False).hrs_prog.sum().reset_index()
programacion_anonimizada


> 🟡 *Muchas veces se confunde la desidentificación con la anonimización,
pero la desidentificación es sólo una técnica de un conjunto de procedimientos a aplicar a la base de datos y, aplicada por si sola, genera un conjunto de datos que puede ser identificable al combinarlos con otros datos de acceso público. Este proceso se denomina *reidentificación* y es lo que se busca
evitar con la anonimización.*

> 🔴 *Si la base de datos contiene variables registradas en texto libre pueden existir identificadores explícitos contenidos en estos campos que no hayan sido reconocidos. Esta posibilidad aumenta cuando el volumen de información es masivo, por lo que estas variables también deben eliminarse previo al proceso de publicación como datos abiertos, mientras no se pueda asegurar la anonimización del campo.*

In [ ]:
#@title Exportar

# Carpeta de destino
datos_crudos = '/content/datos_crudos/'
if not os.path.exists(datos_crudos): os.makedirs(datos_crudos)

# Crear un ExcelWriter para exportar a múltiples hojas
with pd.ExcelWriter(datos_crudos + "Programacion.xlsx") as writer:
    programacion.to_excel(writer, sheet_name="PM", index=False)
    programacion_desidentificada.to_excel(writer, sheet_name="PM_Pseudoanonimizada", index=False)
    programacion_anonimizada.to_excel(writer, sheet_name="PM_Anonimizada", index=False)

print(f"Dataframes exportados a {datos_crudos}Programacion.xlsx ✅")

# Ejemplo 2: Interconsultas No GES

In [ ]:
# @title Cargar planilla

# concatenamos para definir la dirección excel_url, la cual almacena la dirección del archivo a analizar.
datos_originales = 'https://raw.githubusercontent.com/rlagosb/taller_eiv/main/data/'
excel_url = datos_originales + 'SS_MSOr_Respuesta%20Solicitud%20Folio%20AO012T0001655%20(CNE%202021).xlsx'

# importamos desde esa url las hojas del excel LE ABIERTA y CNE EGRESOS
sic_df={}
for hoja in ['LE ABIERTA ','CNE EGRESOS']:
  sic_df[hoja]=pd.read_excel(excel_url, sheet_name=hoja)
  print(f'Hoja {hoja}: {sic_df[hoja].shape[0]} registros')


# Crear tabla única
sic = pd.concat(sic_df.values())

print(sic.info())
sic

🔴 Esta base no tiene identificadores explícitos pero tiene cuasi-identificadores. No debiera existir una combinación única de cuasi-identificadores entre todas las filas de la base de datos 🔍

In [ ]:
sic['EDAD'] = (sic['F_ENTRADA'] - pd.to_datetime(sic['FECHA_NAC'])).dt.days // 365
sic[['EDAD','SEXO']].value_counts(dropna=False)

🟢 **Generalización**: El objetivo de este procedimiento es obtener una k-anonimidad mayor a 1, donde el valor K corresponde a cuantas veces es lo mínimo que está repetido una tupla de cuasi-identificadores en el conjunto de datos. Para lograr esto se limita la precisión de los datos a través del establecimiento de una jerarquía en la que ciertos atributos del mismo grupo comparten valores.

In [ ]:
# Para obtener una K-anonimidad mayor a 2 se aplicó una generalización de la variable edad por rangos etarios

sic['GRUPO_EDAD'] = pd.cut(sic['EDAD'], bins=range(0, 101, 10), right=False)
sic[['GRUPO_EDAD','SEXO']].value_counts(dropna=False)

In [ ]:
# Otra opción es enmascarar las variables para lograr una K-anonimidad mayor a 2
# Enmascaremos la variable Sexo con el valor 99 = desconocido en los casos con k-anonimidad=1

sic.loc[(sic.EDAD.between(60, 70, inclusive='left') & (sic.SEXO==3)), 'SEXO'] = 99
sic.loc[((sic.EDAD>=90) & (sic.SEXO==1)), 'SEXO'] = 99

sic[['GRUPO_EDAD','SEXO']].value_counts(dropna=False)

In [ ]:
# Eliminando la edad quedamos un conjunto de datos k-anonimizados (k=2)
del sic['EDAD']

> *Una vez obtenida una K-anonimidad igual o mayor a 2, se debe evaluar la L- diversidad que corresponde al número de valores distintos de los atributos sensibles que existen en una misma tupla única de cuasi-identificadores. Esto se debe a que si una tupla repetida K veces tiene el mismo atributo
sensible se puede identificar en la base de datos. Al igual que en la K-anonimidad, el valor de la L diversidad debe ser mayor a 1.*

## Exposición de datos sensibles
Consideremos que `PRESTA_MIN` es proxy del diagnóstico del paciente y, por lo tanto, es un atributo sensible (*aquellos datos personales que se refieren a las características físicas o morales de las personas o a hechos o circunstancias de su vida privada o intimidad*)



In [ ]:
# Examinemos si los grupos de cuasi-identificadores tienen l-diversidad > 1

l_diversity_grupos = sic.groupby(['GRUPO_EDAD', 'SEXO'], observed=True)['PRESTA_MIN'].nunique().reset_index().rename(columns={'PRESTA_MIN': 'L_Diversidad'})

print("L-Diversity por grupo:\n", l_diversity_grupos)

if (l_diversity_grupos['L_Diversidad'] < 2).any():
    print("Grupos sin L-diversidad ≥ 2:\n", l_diversity_grupos[l_diversity_grupos['L_Diversidad'] < 2])